<a href="https://colab.research.google.com/github/RexPersicus/ChatGPT_Prompt_Eng_01/blob/main/Claude_multiRAG_LngGrph_mlti_agn_NaturalAdvisor01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#==============================================================================================================================
# This code takes multiple RAG files about different aspects of a subject and can search internet for more context and
# creates a recpmmendation / report.
#
# It needs the RAG files to be text or pdf and they need to be in a folder called uploads.
# It also needs the .env file.
#
# It uses LangGraph, Tavily, Open AI.
#===============================================================================================================================

# Install required packages
!pip install python-dotenv langchain langchain-openai langchain-community tiktoken langgraph openai tavily-python chromadb python-magic PyPDF2 docx2txt

In [2]:
#=======================================================================================
import os
from dotenv import load_dotenv
from typing import Dict, TypedDict, Annotated, Sequence, List
from tavily import TavilyClient
from langgraph.graph import StateGraph
from langchain_core.messages import ToolMessage, HumanMessage, AIMessage, BaseMessage
from langchain_openai import ChatOpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter
import json
import glob
import PyPDF2
import docx2txt
import magic
import operator
from operator import add
from functools import partial

In [3]:
#===================================================================================
# Load environment variables
load_dotenv()

# Initialize API clients
openai_api_key = os.getenv("OPENAI_API_KEY")
tavily_api_key = os.getenv("TAVILY_API_KEY")
tavily_client = TavilyClient(api_key=tavily_api_key)

# Initialize LLM
llm = ChatOpenAI(
    model="gpt-4-turbo-preview",
    temperature=0.7,
    api_key=openai_api_key
)

In [4]:
#===================================================================================
# Define a setter function for non-list values
def set_value(_, new_value):
    return new_value

class HealthAdvisoryState(TypedDict):
    messages: Annotated[List[Dict], add]
    query_topic: Annotated[str, set_value]
    research: Annotated[Dict, set_value]
    rag_results: Annotated[List, set_value]
    analysis: Annotated[str, set_value]
    recommendation: Annotated[str, set_value]
    action_plan: Annotated[str, set_value]

In [5]:
#===================================================================================
class DocumentProcessor:
    """Handles reading and processing different types of health and wellness documents"""

    @staticmethod
    def read_text_file(file_path: str) -> str:
        with open(file_path, 'r', encoding='utf-8') as f:
            return f.read()

    @staticmethod
    def read_pdf_file(file_path: str) -> str:
        text = ""
        with open(file_path, 'rb') as f:
            pdf_reader = PyPDF2.PdfReader(f)
            for page in pdf_reader.pages:
                text += page.extract_text() + "\n"
        return text

    @staticmethod
    def read_docx_file(file_path: str) -> str:
        return docx2txt.process(file_path)

    @staticmethod
    def get_file_type(file_path: str) -> str:
        mime = magic.Magic(mime=True)
        file_type = mime.from_file(file_path)
        return file_type

    @classmethod
    def process_file(cls, file_path: str) -> str:
        file_type = cls.get_file_type(file_path)

        if 'text/plain' in file_type:
            return cls.read_text_file(file_path)
        elif 'application/pdf' in file_type:
            return cls.read_pdf_file(file_path)
        elif 'application/vnd.openxmlformats-officedocument.wordprocessingml.document' in file_type:
            return cls.read_docx_file(file_path)
        else:
            raise ValueError(f"Unsupported file type: {file_type}")

In [13]:
#===================================================================================
def initialize_rag() -> Chroma:
    """Initialize RAG system for health and wellness documents"""
    uploads_path = 'uploads'

    if not os.path.exists(uploads_path):
        os.makedirs(uploads_path)
        print(f"Created {uploads_path} directory. Please add your health and wellness documents there.")
        return None

    files = glob.glob(os.path.join(uploads_path, '*'))

    if not files:
        print(f"No files found in {uploads_path} directory. Please add your health and wellness documents.")
        return None

    all_texts = []
    doc_processor = DocumentProcessor()

    print("Processing health and wellness documents:")
    for file_path in files:
        try:
            print(f"Reading {os.path.basename(file_path)}...")
            text = doc_processor.process_file(file_path)
            all_texts.append(text)
        except Exception as e:
            print(f"Error processing {file_path}: {str(e)}")

    if not all_texts:
        print("No valid documents were processed.")
        return None

    text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    chunks = []
    for text in all_texts:
        chunks.extend(text_splitter.split_text(text))

    embeddings = OpenAIEmbeddings(api_key=openai_api_key)
    vectorstore = Chroma.from_texts(chunks, embeddings)

    print(f"Successfully processed {len(files)} documents into {len(chunks)} chunks")
    return vectorstore

def research_agent(state: HealthAdvisoryState):
    """Research agent that gathers current scientific information about the health topic"""
    print(f"\n🔍 Research Agent: Starting research on {state['query_topic']}...")

    try:
        search_results = tavily_client.search(
            query=f"{state['query_topic']} scientific research natural remedies evidence based studies recent findings",
            search_depth="advanced"
        )
        print("✅ Research Agent: Successfully gathered scientific information")

        return {
            "messages": state["messages"] + [
                {"role": "assistant", "content": f"Research completed for {state['query_topic']}"}
            ],
            "query_topic": state["query_topic"],
            "research": search_results,
            "rag_results": state["rag_results"],
            "analysis": state["analysis"],
            "recommendation": state["recommendation"],
            "action_plan": state["action_plan"]
        }
    except Exception as e:
        print(f"❌ Research Agent Error: {str(e)}")
        return {
            "messages": state["messages"] + [
                {"role": "assistant", "content": f"Error during research: {str(e)}"}
            ],
            "query_topic": state["query_topic"],
            "research": {},
            "rag_results": state["rag_results"],
            "analysis": state["analysis"],
            "recommendation": state["recommendation"],
            "action_plan": state["action_plan"]
        }

In [14]:
def analysis_agent(state: HealthAdvisoryState):
    """Analysis agent that combines research with traditional knowledge"""
    global vectorstore
    print(f"\n🤔 Analysis Agent: Starting analysis of {state['query_topic']}...")

    try:
        # First get how many documents we actually have
        current_docs = len(vectorstore.get()['ids'])
        k = min(5, current_docs)  # Increased to 5 documents for broader health context

        health_info = vectorstore.similarity_search(
            f"natural remedies treatments and approaches for {state['query_topic']}",
            k=k
        )
        print(f"📚 Analysis Agent: Retrieved {k} relevant health document(s)")
    except Exception as e:
        print(f"⚠️ Warning when retrieving documents: {str(e)}")
        health_info = []

    analysis_prompt = f"""
    Based on the following scientific research about {state['query_topic']}:
    {json.dumps(state['research'])}

    And our traditional health knowledge base:
    {health_info}

    Provide a comprehensive analysis that:
    1. Summarizes the current scientific understanding
    2. Identifies traditional natural approaches
    3. Evaluates the evidence for different treatments
    4. Considers potential interactions and contraindications
    5. Highlights safety considerations and precautions

    Include relevant citations and note where evidence is preliminary or limited.
    """

    print("🔄 Analysis Agent: Generating comprehensive analysis...")
    analysis_result = llm.invoke(analysis_prompt)
    print("✅ Analysis Agent: Analysis completed")

    return {
        "messages": state["messages"] + [
            {"role": "assistant", "content": "Analysis completed"}
        ],
        "query_topic": state["query_topic"],
        "research": state["research"],
        "rag_results": health_info,
        "analysis": analysis_result.content,
        "recommendation": state["recommendation"],
        "action_plan": state["action_plan"]
    }

def recommendation_agent(state: HealthAdvisoryState):
    """Agent that creates detailed health recommendations"""
    print("\n✍️ Recommendation Agent: Starting recommendation creation...")

    print(f"Debug - Analysis available: {bool(state['analysis'])}")
    print(f"Debug - Analysis content preview: {state['analysis'][:200] if state['analysis'] else 'No analysis'}")

    recommendation_prompt = f"""
    Create a detailed health advisory report for {state['query_topic']}.
    Use the following analysis: {state['analysis']}

    Format the recommendation with:
    1. Executive Summary
    2. Current Scientific Understanding
    3. Natural Approaches and Traditional Wisdom
    4. Evidence-Based Recommendations
    5. Safety Considerations and Warnings
    6. Lifestyle Integration Suggestions
    7. When to Seek Professional Medical Care

    Important: Include a clear disclaimer about consulting healthcare professionals.
    Make it comprehensive yet accessible to general readers.
    """

    print("🔄 Recommendation Agent: Writing health advisory...")
    recommendation = llm.invoke(recommendation_prompt)
    print(f"Debug - Recommendation generated: {bool(recommendation.content)}")
    print(f"Debug - Recommendation preview: {recommendation.content[:200] if recommendation.content else 'No content'}")

    return {
        "messages": state["messages"] + [
            {"role": "assistant", "content": "Health recommendations generated"}
        ],
        "query_topic": state["query_topic"],
        "research": state["research"],
        "rag_results": state["rag_results"],
        "analysis": state["analysis"],
        "recommendation": recommendation.content,
        "action_plan": state["action_plan"]
    }

def action_plan_agent(state: HealthAdvisoryState):
    """Agent that creates practical implementation steps"""
    print("\n📋 Action Plan Agent: Starting action plan creation...")

    action_plan_prompt = f"""
    Based on the health recommendations for {state['query_topic']}:
    {state['recommendation']}

    Create a practical action plan that includes:
    1. Daily/Weekly Implementation Schedule
    2. Required Resources and Supplies
    3. Preparation Instructions
    4. Progress Tracking Methods
    5. Potential Challenges and Solutions
    6. Adjustment Guidelines
    7. Success Indicators

    Make it practical, achievable, and easy to follow.
    Include relevant safety reminders and precautions.
    """

    print("🔄 Action Plan Agent: Crafting implementation plan...")
    action_plan = llm.invoke(action_plan_prompt)
    print("✅ Action Plan Agent: Implementation plan completed")

    return {
        "messages": state["messages"] + [
            {"role": "assistant", "content": "Action plan generated"}
        ],
        "query_topic": state["query_topic"],
        "research": state["research"],
        "rag_results": state["rag_results"],
        "analysis": state["analysis"],
        "recommendation": state["recommendation"],
        "action_plan": action_plan.content
    }

In [15]:
#========================================================================================
def build_graph():
    """Build the LangGraph workflow for health advisory system"""
    workflow = StateGraph(HealthAdvisoryState)

    # Add nodes
    workflow.add_node("research_node", research_agent)
    workflow.add_node("analysis_node", analysis_agent)
    workflow.add_node("recommendation_node", recommendation_agent)
    workflow.add_node("action_plan_node", action_plan_agent)

    # Define edges - Sequential flow of analysis
    workflow.add_edge("research_node", "analysis_node")
    workflow.add_edge("analysis_node", "recommendation_node")
    workflow.add_edge("recommendation_node", "action_plan_node")

    # Set entry and exit points
    workflow.set_entry_point("research_node")
    workflow.set_finish_point("action_plan_node")

    return workflow.compile()

In [16]:
#===================================================================================

def run_health_advisor():
    """Main application loop"""
    global vectorstore
    print("Initializing Natural Health Advisory System...")
    vectorstore = initialize_rag()

    if vectorstore is None:
        print("Please add health and wellness documents to the 'uploads' folder and restart the application.")
        return

    while True:
        print("\n" + "="*50)
        print("""
Welcome to the Natural Health Advisory System
Please enter your health-related question or topic.
Examples:
- Natural remedies for chronic inflammation
- Holistic approaches to stress management
- Herbal supplements for immune support
        """)

        query_topic = input("\nEnter your health question or topic: ")

        # Add initial disclaimer
        print("""
IMPORTANT DISCLAIMER:
This system provides educational information about natural health approaches.
It is not a substitute for professional medical advice, diagnosis, or treatment.
Always seek the advice of your physician or other qualified health provider.
        """)

        graph = build_graph()

        # Create initial state
        initial_state = {
            "messages": [{
                "content": f"Starting analysis for: {query_topic}",
                "role": "user",
            }],
            "query_topic": query_topic,
            "research": {},
            "rag_results": [],
            "analysis": "",
            "recommendation": "",
            "action_plan": ""
        }

        try:
            result = graph.invoke(initial_state)

            # Display recommendation with better error handling
            print("\n=== Health Advisory Report ===")
            if result and "recommendation" in result and result["recommendation"]:
                print("\n" + result["recommendation"])
            else:
                print("No health recommendations were generated.")
                print(f"Debug - Result keys: {result.keys() if result else 'No result'}")

            show_action_plan = input("\nWould you like to see the detailed action plan? (yes/no): ")
            if show_action_plan.lower() == 'yes':
                print("\n=== Implementation Action Plan ===")
                if result and "action_plan" in result and result["action_plan"]:
                    print("\n" + result["action_plan"])
                else:
                    print("No action plan was generated.")

            # Final reminder
            print("""
REMINDER: The information provided is for educational purposes only.
Consult with healthcare professionals before making any changes to your health routine.
            """)

        except Exception as e:
            print(f"Error during execution: {str(e)}")

        another = input("\nWould you like to explore another health topic? (yes/no): ")
        if another.lower() != 'yes':
            break

In [17]:
#====================================================================================

if __name__ == "__main__":
    run_health_advisor()

Initializing Natural Health Advisory System...
Processing health and wellness documents:
Reading 03_RAG_Natural_Health.pdf...
Reading 01_RAG_Natural_Health.pdf...
Reading 04_RAG_Natural_Health.pdf...
Reading 02_RAG_Natural_Health.pdf...


<ipython-input-13-86042287550b>:38: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings(api_key=openai_api_key)


Successfully processed 4 documents into 4 chunks


Welcome to the Natural Health Advisory System
Please enter your health-related question or topic.
Examples:
- Natural remedies for chronic inflammation
- Holistic approaches to stress management
- Herbal supplements for immune support
        

Enter your health question or topic: How to fix Eczema?

IMPORTANT DISCLAIMER:
This system provides educational information about natural health approaches.
It is not a substitute for professional medical advice, diagnosis, or treatment.
Always seek the advice of your physician or other qualified health provider.
        

🔍 Research Agent: Starting research on How to fix Eczema?...
✅ Research Agent: Successfully gathered scientific information

🤔 Analysis Agent: Starting analysis of How to fix Eczema?...
📚 Analysis Agent: Retrieved 4 relevant health document(s)
🔄 Analysis Agent: Generating comprehensive analysis...
✅ Analysis Agent: Analysis completed

✍️ Recommendation Agent: Starting recommen